### 背景
分析全球和自己所在地的气温数据，比较所在城市的气温走向与全球气温走向。

### 数据获取
从数据库中提取数据。通过[Udacity](https://classroom.udacity.com/nanodegrees/nd002-cn-basic-vip/parts/d1865612-f3fd-4db0-80c7-348c594d573d/modules/7b83f9fd-759a-4cc6-8456-ce3783e17475/lessons/dce89631-d141-4a36-b3fd-5e8ec038bc70/concepts/530f21c0-2f37-4390-aaab-3ce440e56d80)提供的工作区，该工作区与数据库连接。

提取出两个.csv文件：

- results_china.csv 包含中国所有城市的平均气温数据
- results_global.csv 包含全球的平均气温数据

### 交互式可视化
使用Python将提取的数据可视化成一个线条图，用户可选择不同城市和移动N值来比较不同城市和全球的气温趋势。

#### 说明

1. 为了实现下拉框交互式选择城市功能，需要从`results_china.csv`数据中过滤出对应城市的数据。------> 函数filter_city()

   1）对城市数据的`avg_temp`列的缺失值作处理
   
   2）按年份，将城市的气温数据和全球的气温数据合并放到一个列表


2. 为了使绘制的线图更加平滑，便于观察气温走向，所以采用气温的移动平均值，而不是原始的年平均值。因而需要先计算移动平均值----->函数`calculate_moving_average()`

   1）移动平均值的计算方法这里采用[pandas.rolling_mean](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.rolling_mean.html)函数。但是，按照文档说明传入参数后，会报出`AttributeError: module 'pandas' has no attribute 'rolling_mean'
`错误，后来发现是`pandas`库的版本问题，具体解决方案可参考[这里](https://stackoverflow.com/questions/50482884/module-pandas-has-no-attribute-rolling-mean)。

   2）移动平均值窗口大小的设定需要权衡考虑。设置得过小，起不到平滑的作用，波动仍然会很剧烈；设置得过大，数据越平滑，但是准确性和敏感性就降低的越多，所以需要在数据变化准确性和平滑程度之间进行一个权衡。这里用户可根据不同的分析目的自由选择移动N值的值，范围在2到15之间。


3. 有了数据后，就可以开始绘制图形了----->函数`show_fig()`

   1）绘制图使用`matplotlib.pyplot`模块；设置坐标轴的ticks需用到`matplotlib.ticker`模块；matplotlib绘图可视化的各属性设置可参考[这里](http://python.jobbole.com/85106/)

In [8]:
"""分析所在城市和全球的气温趋势
"""
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline



def calculate_moving_average(data,window):
    """计算移动平均值
       参数1：需要计算移动平均值的数据
       参数2：移动窗口大小
    """
    data['city_mavg'] = round(data['avg_temp_x'].rolling(window).mean(),2)
    data['global_mavg'] = round(data['avg_temp_y'].rolling(window).mean(),2)
    return data


def show_fig(data):
    """将所在城市与全球平均气温数据绘制成折线图并展示
       参数1：城市平均气温数据
       参数2：全球平均气温数据
    """
    #设置显示的图片大小
    figsize = 10,5 
    figure, ax = plt.subplots(figsize=figsize) 
    
    #X,Y轴的标签
    plt.xlabel('Year')
    plt.ylabel('Moving Average Temperature (ºC)')

    xmajorLocator   = MultipleLocator(20) #将x轴主刻度标签设置为20的倍数
    xminorLocator   = MultipleLocator(5) #将x轴次刻度标签设置为5的倍数
    ax.xaxis.set_major_locator(xmajorLocator)
    ax.xaxis.set_minor_locator(xminorLocator)


    ymajorLocator   = MultipleLocator(1) #将y轴主刻度标签设置为1的倍数
    yminorLocator   = MultipleLocator(0.2) #将y轴次刻度标签设置为0.2的倍数
    ax.yaxis.set_major_locator(ymajorLocator)
    ax.yaxis.set_minor_locator(yminorLocator)
    
    ax.xaxis.grid(True, which='major') #x坐标轴的网格使用主刻度
    ax.yaxis.grid(True, which='minor') #y坐标轴的网格使用次刻度


    plt.plot(data['year'],data['city_mavg'],color='skyblue',linewidth='2',label=data['city'][0])
    plt.plot(data['year'],data['global_mavg'],color='lightgreen',linewidth='2',label='Global')
    plt.legend()
    plt.title('Temperature Trend')

    plt.show() 


def filter_city(city):
    """从results_china文件中过滤出某一个城市，处理缺失值之后将该城市的气温数据与全球气温数据合在一起
       参数：城市名
    """
    china_data = pd.read_csv('data/results_china.csv')
    global_data = pd.read_csv('data/results_global.csv')
    city_data = china_data[china_data.city == city]
    city_data = city_data.sort_values(by='year')
    
    try:
        year_nonull = int(city_data[city_data.avg_temp.isnull()].tail(1).year) # 查找avg_temp列缺失的年份，将其记录在year_nonull中
    except:
        data = city_data.merge(global_data,how='left',on='year') # 异常：year_nonull不存在
    else:
        city_data = city_data[city_data.year > year_nonull]  # year_nonull存在
        data = city_data.merge(global_data,how='left',on='year')

    return data
# print(filter_city('Wuhan'))


def project_func(city,x):
    window = x
    data = calculate_moving_average(filter_city(city),window)
    show_fig(data)

    
city_origin={
    '昆明':'Kunming','齐齐哈尔':'Qiqihar','西安':'Xian',
    '兰州':'Lanzhou','天津':'Tianjin','成都':'Chengdu',
    '唐山':'Tangshan','大同':'Datong','太原':'Taiyuan',
    '哈尔滨':'Harbin','长春':'Changchun','沈阳':'Shenyang',
    '大连':'Dalian','鞍山':'Anshan','吉林':'Jilin',
    '南京':'Nanning','济南':'Jinan','邯郸':'Handan',
    '广州':'Guangzhou','贵阳':'Guiyang','洛阳':'Luoyang',
    '佛山':'Foshan','徐州':'Xuzhou','无锡':'Wuxi',
    '合肥':'Hefei','杭州':'Hangzhou','苏州':'Suzhou',
    '常州':'Changzhou','南昌':'Nanchang','青岛':'Qingdao',
    '福州':'Fuzhou','武汉':'Wuhan','南京':'Nanjing','上海':'Shanghai'    
}
    
interact(project_func, city=city_origin, x=widgets.IntSlider(min=2,max=20,description='移动N值'))


interactive(children=(Dropdown(description='city', options={'昆明': 'Kunming', '齐齐哈尔': 'Qiqihar', '西安': 'Xian', …

<function __main__.project_func(city, x)>